In [ ]:
!pip install sentence-transformers faiss-cpu openai pdfplumber -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 121.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 104.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.

In [ ]:
import pdfplumber
import re
import json

def parse_georgian_criminal_code(pdf_path):
    data = []
    current_chapter = None
    current_article = None
    current_article_text = ""

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if not text:
                continue

            lines = text.split('\n')
            for line in lines:
                line = line.strip()

                # Detect new chapter
                chapter_match = re.match(r'^თავი\s+[IVXLCDM\d]+', line)
                if chapter_match:
                    current_chapter = line
                    continue

                # Detect article start
                article_match = re.match(r'^მუხლი\s+(\d+)[.]*\s*(.+)?', line)
                if article_match:
                    # Lets Save previous article if it exists
                    if current_article:
                        data.append({
                            "თავი": current_chapter,
                            "მუხლი": current_article,
                            "ტექსტი": current_article_text.strip()
                        })
                    current_article = f"{article_match.group(1)}. {article_match.group(2) or ''}".strip()
                    current_article_text = ""
                    continue

                # Accumulate article body
                if current_article:
                    current_article_text += line + " "

    # Save last article
    if current_article:
        data.append({
            "თავი": current_chapter,
            "მუხლი": current_article,
            "ტექსტი": current_article_text.strip()
        })

    return data

# Running
pdf_path = "/content/Criminal Code of Georgia.pdf"  # change to your file path
parsed_articles = parse_georgian_criminal_code(pdf_path)

# Saving to JSON
with open("parsed_criminal_code.json", "w", encoding="utf-8") as f:
    json.dump(parsed_articles, f, ensure_ascii=False, indent=2)


In [ ]:
import json


with open("parsed_criminal_code.json", "r", encoding="utf-8") as f:
    raw_articles = json.load(f)

# Converting to format
formatted_articles = []

for item in raw_articles:
    article_text = item["ტექსტი"].strip()
    article_number = item["მუხლი"].split(".")[0].strip()  # e.g., "1"
    article_title = item["მუხლი"].strip()

    formatted_articles.append({
        "article": f"მუხლი {article_number}",
        "text": article_text
    })

# Save
with open("criminal_code_for_embeddings.json", "w", encoding="utf-8") as f:
    json.dump(formatted_articles, f, ensure_ascii=False, indent=2)

print("✅ Reformatted and saved for embedding.")


✅ Reformatted and saved for embedding.


In [ ]:
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer


model = SentenceTransformer("BAAI/bge-m3")


with open("criminal_code_for_embeddings.json", "r", encoding="utf-8") as f:
    data = json.load(f)

#Extracting texts to embed
texts = [item["text"] for item in data]

#Embedding
print("🔄 Generating embeddings...")
embeddings = model.encode(texts, show_progress_bar=True, normalize_embeddings=True)
embeddings = np.array(embeddings).astype("float32")

#Creating index and add embeddings
print("🧠 Creating FAISS index...")
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

#Saving index and metadata
faiss.write_index(index, "criminal_code.index")
with open("criminal_code_metadata.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print("✅ Embeddings created and saved.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

🔄 Generating embeddings...


Batches:   0%|          | 0/16 [00:00<?, ?it/s]

🧠 Creating FAISS index...
✅ Embeddings created and saved.


In [ ]:
import faiss
import json


index = faiss.read_index("criminal_code.index")

# Loading original texts (will be using this for retrieval later)
with open("criminal_code_for_embeddings.json", "r", encoding="utf-8") as f:
    texts = json.load(f)

print(f"Index contains {index.ntotal} entries")


Index contains 496 entries


In [ ]:
!pip install -U langchain langchain-community langchain-openai langchain-huggingface -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00


In [ ]:
!pip install langchain-google-genai google-generativeai -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.4 MB/s eta 0:00:00


In [ ]:
!pip install rank_bm25

In [ ]:
import os

os.environ["GOOGLE_API_KEY"] = ""

In [ ]:
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA
from langchain.docstore import InMemoryDocstore
from rank_bm25 import BM25Okapi
import json
import faiss
import os
import getpass

def setup_google_api_key():
    """Setup Google API key with proper validation"""
    if "GOOGLE_API_KEY" not in os.environ:
        print("Google API key not found in environment variables.")
        print("Please get your API key from: https://makersuite.google.com/app/apikey")
        api_key = getpass.getpass("Enter your Google API key: ")
        os.environ["GOOGLE_API_KEY"] = api_key
    else:
        print("Using Google API key from environment variables.")


def is_query_unserious(query, llm):

    prompt = f"""
დარწმუნებული ხარ 90%-ით მაინც, რომ შემდეგი კითხვა ხუმრობითა ან არასერიოზულად არის დაწერილი?

მხოლოდ ერთი სიტყვით მიპასუხე: "დიახ" ან "არა"

კითხვა:
"{query}"
"""
    try:
        response = llm.invoke(prompt)

        # Handling both string and AIMessage
        if hasattr(response, "content"):
            content = response.content.strip().lower()
        else:
            content = str(response).strip().lower()

        print(f"🤖 დეტექცია ხუმრობისთვის: {content}")

        return "დიახ" in content

    except Exception as e:
        print(f"⚠️ ვერ მოხერხდა ხუმრობის განსაზღვრა: {e}")
        return False

def load_qa_system():
    setup_google_api_key()

    print("🔠 Loading embedding model...")
    embedding_model = HuggingFaceEmbeddings(
        model_name="BAAI/bge-m3",
        model_kwargs={"device": "cpu"}
    )

    if not os.path.exists("criminal_code.index"):
        raise FileNotFoundError("criminal_code.index file not found!")

    if not os.path.exists("criminal_code_metadata.json"):
        raise FileNotFoundError("criminal_code_metadata.json file not found!")

    print("📄 Loading FAISS index and metadata...")
    index = faiss.read_index("criminal_code.index")

    with open("criminal_code_metadata.json", "r", encoding="utf-8") as f:
        metadata = json.load(f)

    print(f"📄 Rebuilding {len(metadata)} documents...")
    documents = [
        Document(
            page_content=item["text"],
            metadata={"source": f"მუხლი {item.get('article', 'N/A')}"}
        )
        for item in metadata
    ]

    # FAISS vector store
    docstore = InMemoryDocstore({str(i): doc for i, doc in enumerate(documents)})
    vectorstore = FAISS(
        index=index,
        embedding_function=embedding_model,
        docstore=docstore,
        index_to_docstore_id={i: str(i) for i in range(len(documents))}
    )
    vector_retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

    # BM25 keyword search setup
    bm25_tokenized = [doc.page_content.lower().split() for doc in documents]
    bm25 = BM25Okapi(bm25_tokenized)

    # Loading Google LLM
    models_to_try = [
        "gemini-2.5-flash",
        "gemini-2.0-flash",
        "gemini-1.5-flash"
    ]
    llm = None
    for model_name in models_to_try:
        try:
            print(f"🤖 Trying to load model: {model_name}")
            llm = ChatGoogleGenerativeAI(
                model=model_name,
                temperature=0,
                convert_system_message_to_human=True
            )
            llm.invoke("Hello")
            print(f"✅ Successfully loaded model: {model_name}")
            break
        except Exception as e:
            print(f"❌ Failed to load {model_name}: {str(e)}")

    if llm is None:
        raise Exception("Could not load any Google LLM.")

    # ✅ Defining qa_chain before return
    def qa_chain(query, boost_keywords=[]):

        vector_docs = vector_retriever.get_relevant_documents(query)

        # Keyword search with boosting
        query_tokens = query.lower().split()
        if boost_keywords:
            print("🔎 პრიორიტეტული სიტყვებით გაძლიერებული ძიება:", boost_keywords)
            query_tokens += boost_keywords * 5

        bm25_scores = bm25.get_scores(query_tokens)
        top_bm25_indices = sorted(range(len(bm25_scores)), key=lambda i: bm25_scores[i], reverse=True)[:5]
        keyword_docs = [documents[i] for i in top_bm25_indices]

        # Combine and deduplicate
        combined_docs = vector_docs + keyword_docs
        unique_docs = list({doc.page_content: doc for doc in combined_docs}.values())

        # Final LLM prompt with top 3
        context = "\n\n".join([doc.page_content for doc in unique_docs[:3]])
        prompt = f"""
კითხვა: {query}

გამოიყენე შემდეგი კანონმდებლობითი ამონარიდები პასუხისთვის:

{context}

პასუხი ქართული ენით:
"""
        response = llm.invoke(prompt)
        return response.content, unique_docs[:3]

    #Final
    return qa_chain, llm



def detect_prior_conviction_risk(query, llm):
    prompt = f"""
შეაფასე, შესაძლებელია თუ არა, რომ მოცემული კითხვა ეხება პირს, რომელიც ადრე ნასამართლევი იყო?

თუ დარწმუნებული ხარ მინიმუმ 90%-ით, მიპასუხე მხოლოდ ერთი სიტყვით: დიახ ან არა.

კითხვა:
"{query}"
"""
    try:
        response = llm.invoke(prompt)
        content = response.content.strip().lower() if hasattr(response, "content") else str(response).strip().lower()
        return "დიახ" in content
    except Exception as e:
        print(f"⚠️ ვერ მოხერხდა ნასამართლეობის სეგმენტის განსაზღვრა: {e}")
        return False


def user_confirmed_prior_conviction(answer_text, llm):
    prompt = f"""
შემდეგი პასუხი იმაზე მიანიშნებს თუ არა, რომ მოქმედი პირი ადრე მართლაც ნასამართლევი იყო?

პასუხი: "{answer_text}"

მიპასუხე მხოლოდ ერთი სიტყვით: დიახ ან არა.
"""
    try:
        response = llm.invoke(prompt)
        content = response.content.strip().lower() if hasattr(response, "content") else str(response).strip().lower()
        return "დიახ" in content
    except Exception as e:
        print(f"⚠️ ვერ მოხდა პასუხის ინტერპრეტაცია: {e}")
        return False


def ask_question(qa_chain, query, llm):
    try:
        print(f"\n❓ კითხვა: {query}")
        print("დამუშავება მიმდინარეობს...")

        unserious = is_query_unserious(query, llm)
        boost_keywords = []

        #Checking if question is possibly about prior conviction
        prior_check = detect_prior_conviction_risk(query, llm)
        if prior_check:
            print("\n🤖 გამარჯობა. ვეცდები დაგეხმარო, თუმცა მანამდე მაინტერესებს, მოქმედი პირი ოდესმე ნასამართლევი ხომ არ ყოფილა?")
            user_reply = input("➡️ შეიყვანე პასუხი: ").strip()
            if user_confirmed_prior_conviction(user_reply, llm):
                boost_keywords = ["ნასამართლევი", "განმეორებით", "ხელმეორედ"]

        #Generating answer
        result, source_docs = qa_chain(query, boost_keywords=boost_keywords)

        if unserious:
            result = "🤔 ვიფიქრე, რომ ეს ხუმრობა იყო ან კარგად არ გაქვს გარკვეული ეს საკითხი.\n\n" + result

        print("\n🤖 პასუხი:")
        print(result)

        print("\n📄 წყაროები:")
        for i, doc in enumerate(source_docs, 1):
            print(f"{i}. {doc.metadata['source']}")
            print(f"   ტექსტი: {doc.page_content[:200]}...")
            print()

        return result

    except Exception as e:
        print(f"შეცდომა: {e}")
        return None


def main():
    """Main function to run the QA system"""
    try:
        # Load the QA system
        qa_chain, llm = load_qa_system()

        #Original Question
        query = "რა სასჯელი ეკუთვნის ქურდს?"
        #query = "რა სასჯელი ეკუთვნის მფრინავ თხას?"
        ask_question(qa_chain, query, llm)

        # Interactive mode
        print("\n" + "="*50)
        print("Interactive mode - type 'quit' to exit")
        print("="*50)

        while True:
            user_query = input("\nEnter your question: ").strip()
            if user_query.lower() in ['quit', 'exit', 'q']:
                print("Goodbye!")
                break

            if user_query:
                ask_question(qa_chain, user_query)

    except Exception as e:
        print(f"Error: {str(e)}")
        print("\nTroubleshooting tips:")
        print("1. Make sure you have a valid Google API key")
        print("2. Check that criminal_code.index and criminal_code_metadata.json files exist")
        print("3. Ensure you have sufficient API quota")
        print("4. Check your internet connection")

if __name__ == "__main__":
    main()

Using Google API key from environment variables.
🔠 Loading embedding model...
📄 Loading FAISS index and metadata...
📄 Rebuilding 496 documents...
🤖 Trying to load model: gemini-2.5-flash


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


✅ Successfully loaded model: gemini-2.5-flash

❓ კითხვა: რა სასჯელი ეკუთვნის ქურდს?
დამუშავება მიმდინარეობს...


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


🤖 დეტექცია ხუმრობისთვის: არა


...


🤖 გამარჯობა. ვეცდები დაგეხმარო, თუმცა მანამდე მაინტერესებს, მოქმედი პირი ოდესმე ნასამართლევი ხომ არ ყოფილა?
➡️ შეიყვანე პასუხი: კი, იყო ნასამართლევი


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


🔎 პრიორიტეტული სიტყვებით გაძლიერებული ძიება: ['ნასამართლევი', 'განმეორებით', 'ხელმეორედ']


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



🤖 პასუხი:
საქართველოს კანონმდებლობის მიხედვით, ქურდისთვის განკუთვნილი სასჯელი დამოკიდებულია ქურდობის სახეზე, ჩადენის გარემოებებზე და მიყენებული ზიანის ოდენობაზე. ასევე, ცალკეა გათვალისწინებული სასჯელი ქურდული სამყაროს წევრობისთვის ან კანონიერი ქურდობისთვის, რაც არ არის უშუალოდ ქურდობის აქტი, არამედ სტატუსი ან ორგანიზებული დანაშაულებრივი საქმიანობა.

მოცემული კანონმდებლობითი ამონარიდების მიხედვით, სასჯელები შემდეგნაირად განისაზღვრება:

**I. ქურდობა (სისხლის სამართლის კოდექსის 177-ე მუხლი)**
ქურდობა არის სხვისი მოძრავი ნივთის ფარული დაუფლება მართლსაწინააღმდეგო მისაკუთრების მიზნით. სასჯელი დამოკიდებულია სიმძიმეზე:

1.  **ძირითადი ქურდობა (177.1):**
    *   **სასჯელი:** ჯარიმა ან თავისუფლების შეზღუდვა ვადით ერთიდან სამ წლამდე ანდა თავისუფლების აღკვეთა იმავე ვადით.

2.  **ქურდობა დამამძიმებელ გარემოებებში (177.2):**
    *   **გარემოებები:**
        *   რამაც მნიშვნელოვანი ზიანი გამოიწვია (ნივთის ღირებულება 150 ლარის ზევით).
        *   ჩადენილი სადგომში ან სხვა საცავში უკანონო შეღწევით.
  